In [133]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from geopy.geocoders import Nominatim
import json
import requests
from math import sqrt
import numpy as np
import rsome as rso
from rsome import ro
from rsome import ort_solver as solver
import folium

### 1. Use webdriver to put website into state of scrapping

In [4]:
baseUrl = "https://www.oeps.at/de/termine"
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('headless')

#Create the webdriver instance
browser = webdriver.Chrome("./chromedriver", options=chrome_options)

#Open the url
browser.get(baseUrl)

#Find element to filter for the whole year data
dropdown = browser.find_element_by_xpath('/html/body/main/div[1]/div/div/div/div[2]/form/span[1]/select[1]')
dropdown.click()

#Click
option = browser.find_element_by_xpath("/html/body/main/div[1]/div/div/div/div[2]/form/span[1]/select[1]/option[1]")
option.click()

#Submit
submit = browser.find_element_by_xpath("/html/body/main/div[1]/div/div/div/div[2]/form/input")
submit.click()

#Get data
all_data = browser.find_element_by_class_name("appointmentslist").text

### 2. Scrap data from website and assemble it into a dataframe

In [5]:
#Convert month names into numerics
months_conversion = {
    "Jänner":1,
    "Februar":2,
    "März":3,
    "April":4,
    "Mai":5,
    "Juni":6,
    "Juli":7,
    "August":8,
    "September":9,
    "Oktober":10,
    "November":11,
    "Dezember":12
}

In [6]:
#Connect bs4 to selenium
html = browser.page_source

#Instantiate soup instance
soup = BeautifulSoup(html, 'lxml')

#Get main appointment list
main_div = soup.find('div', class_='appointmentslist')

#Get all appointments for competitions
all_divs = main_div.find_all("div", class_="appointment")

#Final data
data = []

for test in all_divs:
    
    curr = {}
    
    #Month & date
    full_date = test.find("div", class_="date").text
    
    #Only get the start of the competition to determine the week
    date = " ".join(full_date.split()).split()
    month = months_conversion[date[0]]
    start_date = date[1]
    
    #Tournament category
    category = test.find("div", class_="category").text
    category = category.split()
    
    #We only want showjumping competitions
    category = list(filter(lambda x: "S" in x,category))
    
    #If the list is empty (as we know that S stands for jumping) we drop this competition
    if len(category) == 0:
        continue
        
    category = ", ".join(category)
    
    #Location
    location = test.find("div", class_="location").text.split()
    
    #Only want the city or village of the location
    location = " ".join(location[1:])
    
    #Assemble dictionary
    curr["Month"] = month
    curr["Date"] = start_date
    curr["Destination"] = location
    curr["Category"] = category
    
    #Append to data
    data.append(curr)
    
#Close browser
browser.close()
    
data = pd.DataFrame(data)
    

In [7]:
#Final data after webscraping
data

,Month,Date,Destination,Category
0,1,28.01.2022,Ebreichsdorf,CSN-B*
1,2,11.02.2022,Stadl Paura,CSN-B*
2,2,19.02.2022,Tulln an der Donau,CSN-C
3,2,25.02.2022,Ebreichsdorf,CSN-B*
4,2,25.02.2022,Stadl Paura,CSN-B*
...,...,...,...,...
180,11,04.11.2022,Stadl Paura,CSN-B*
181,11,11.11.2022,Stadl Paura,CSN-B*
182,11,25.11.2022,Ebreichsdorf,"CSN-B*, CSNP-B"
183,12,02.12.2022,Stadl Paura,"CSN-A*, CSN-B"


### 3. Use Maps API to find distance between home base and tournament to get transportation cost

#### The distance is used to calculate the transportation cost if the manager decides to send the riders there

##### First, I map the latitude and longitude to every location and then I find the driving distance between the home base and the location

In [8]:
"""From my previous runs, I saw that some locations cannot be mapped to lat long given by the webiste. 
So I used the actual address of the location or a larger location close by."""

bad_loc = {
    "St. Margarethen-Stückler": "Reiterhofstraße 38, 9412 Wolfsberg, Austria",
    "Auhof Neufeld": "Neufeld an Leitha",
    "Treffen - GHPC": "Gaston Glock Straße 8, 9521 Treffen am Ossiacher See, Austria",
    "Wien-Freudenau/ASKÖ":"Freudenau 555, 1020 Wien, Austria"
}

In [9]:
#My base location was my old horse riding stable
data["Origin"] = "Zwentendorf"
geolocator = Nominatim(user_agent="Leo")
home = geolocator.geocode("Zwentendorf")

#Add lat and long to all destinations
def lat_long(address):
    
    geolocator = Nominatim(user_agent="Leo")
    location = geolocator.geocode(address)
    
    if not location:
        location = geolocator.geocode(bad_loc[address])
    
    return [location.longitude, location.latitude]

#Getting both coords at the same time to limit runtime
data["Destination_Coords"] = data["Destination"].apply(lambda x: lat_long(x))
data["Destination_Long"] = data["Destination_Coords"].apply(lambda x: x[0])
data["Destination_Lat"] = data["Destination_Coords"].apply(lambda x: x[1])
data["Origin_Long"] = home.longitude
data["Origin_Lat"] = home.latitude

In [226]:
data

,Month,Date,Destination,Category,Origin,Destination_Coords,Destination_Long,Destination_Lat,Origin_Lat,Origin_Long,Distance
0,1,28.01.2022,Ebreichsdorf,CSN-B*,Zwentendorf,"[16.39163215125545, 47.96611095]",16.391632,47.966111,48.610603,16.43302,98.7359
1,2,11.02.2022,Stadl Paura,CSN-B*,Zwentendorf,"[13.8694007, 48.0838623]",13.869401,48.083862,48.610603,16.43302,258.7631
2,2,19.02.2022,Tulln an der Donau,CSN-C,Zwentendorf,"[16.0567744, 48.3311686]",16.056774,48.331169,48.610603,16.43302,55.9459
3,2,25.02.2022,Ebreichsdorf,CSN-B*,Zwentendorf,"[16.39163215125545, 47.96611095]",16.391632,47.966111,48.610603,16.43302,98.7359
4,2,25.02.2022,Stadl Paura,CSN-B*,Zwentendorf,"[13.8694007, 48.0838623]",13.869401,48.083862,48.610603,16.43302,258.7631
...,...,...,...,...,...,...,...,...,...,...,...
180,11,04.11.2022,Stadl Paura,CSN-B*,Zwentendorf,"[13.8694007, 48.0838623]",13.869401,48.083862,48.610603,16.43302,258.7631
181,11,11.11.2022,Stadl Paura,CSN-B*,Zwentendorf,"[13.8694007, 48.0838623]",13.869401,48.083862,48.610603,16.43302,258.7631
182,11,25.11.2022,Ebreichsdorf,CSN-B*,Zwentendorf,"[16.39163215125545, 47.96611095]",16.391632,47.966111,48.610603,16.43302,98.7359
183,12,02.12.2022,Stadl Paura,CSN-A*,Zwentendorf,"[13.8694007, 48.0838623]",13.869401,48.083862,48.610603,16.43302,258.7631


In [14]:
data.to_csv("./data/interim.csv",index=False)

In [88]:
def get_drive_distance(long_o, lat_o, long_d, lat_d):
    
    r = requests.get(f"http://router.project-osrm.org/route/v1/car/{long_o},{lat_o};{long_d},{lat_d}?overview=false""")
    
    routes = json.loads(r.content)
    route = routes.get("routes")[0]
    distance = route["distance"]
        
    return distance

In [89]:
data["Distance"] = data.apply(lambda x: get_drive_distance(x.Origin_Long, x.Origin_Lat, x.Destination_Long, x.Destination_Lat), axis=1)


In [92]:
#Given that Distance is given in meters, I readjust
data["Distance"] = data["Distance"] / 1000

In [94]:
data.to_csv("./data/interim.csv",index=False)

### Create cost and revenue paramters

1. Transportation Cost
2. Competition related expenses (depending on category of competition)
3. Energy drain for the horse (depending on category of competition)
4. Revenue is calculated on possible prize money (depending on category of competition)


The cost for transporting hourses to a competition is the distance * the gasoline price per kilometer * fuel efficiency * 2

Fuel Efficiency of trucks (used to transport horses) = 35 litres / kilometer (Source states 30-40)
Source: https://www.webfleet.com/en_gb/webfleet/blog/do-you-know-the-diesel-consumption-of-a-lorry-per-km/

Price of Gasoline in Austria = 1.59 Euro (Average from November 2021 till March 2022)
Source: https://www.globalpetrolprices.com/Austria/Vienna/gasoline_prices/

Cost = Fuel Efficiency * Price

### Category Explanation

Each category of competition has different values associated to it, as it is the case reality.

Terminology of categories:

CSN standards for a national competition so within Austria
Then the ordinality of prize money and cost goes from C to B to A, where A has the highest cost and highest prize money

CSI stands for an international competition within Austria which normally is more expensive and has more prize money than CSN

For CSI the ordinality is similar to CSN but instead of having A, B and C there are stars where 1* has the lowest cost and prize and 4* or 5* has the highest prize and expense.

Under the compete metrics all metrics for competition are 1) pulled from the playbook of the Austrian Horse Association and 2) from personal experience. However, the amount of horse-drain is purely estimated from experience. Horse drain in that is understood as in how much the horse looses energy and as such ability to win with each competition entered.

Entry cost is same as transportation cost a binary cost because it does not matter on the amount of riders or horses the manager intends to send, whereas average_prize and horse_drain depend on the horse



In [373]:
data = pd.read_csv("./data/interim.csv")

In [5]:
compete_metrics = pd.DataFrame([
    {"cat":"CSN-C", "Entry Cost":150, "Average Price":50, "Horse Drain": -10},
    {"cat":"CSN-B", "Entry Cost":200, "Average Price":100, "Horse Drain": -15},
    {"cat":"CSN-B*", "Entry Cost":200, "Average Price":100, "Horse Drain": -15},
    {"cat":"CSN-A","Entry Cost":250, "Average Price":500, "Horse Drain": -25},
    {"cat":"CSN-A*","Entry Cost":250, "Average Price":500, "Horse Drain": -25},
    {"cat":"CSI1*", "Entry Cost":350, "Average Price":500, "Horse Drain": -15},
    {"cat":"CSI2*", "Entry Cost":400, "Average Price":800, "Horse Drain": -25},
    {"cat":"CSI3*", "Entry Cost":450, "Average Price":1000, "Horse Drain": -30},
    {"cat":"CSI4*", "Entry Cost":450, "Average Price":2000,"Horse Drain": -40}
])

cat_order = {
    "CSN-C": 1,
    "CSN-B": 2,
    "CSN-B*": 2,
    "CSN-A": 3,
    "CSN-A*": 3,
    "CSI1*": 4,
    "CSI2*": 5,
    "CSI3*": 6,
    "CSI4*": 7,
}

In [6]:
#I only want the competitions mentioned above and define that if a competition implies multiple categories, then only the highest one is considered

def cat_cleaner(row):
    
    cats = row.split(", ")
    
    if len(cats) < 2:
        
        if row not in cat_order:
            
            return "Drop"
        
        else:
            return row
    
    else:
        cats = list(filter(lambda x: x in cat_order.keys(), cats))
        
        if(len(cats)) < 1:
            return "Drop"
        
        else:
            cats = list(map(lambda x: [x, cat_order[x]], cats))
            cats.sort(reverse=True, key=lambda x: x[1])
            
            return cats[0][0]
        
data["Category"] = data["Category"].apply(lambda x: cat_cleaner(x))
    

In [7]:
data

,Month,Date,Destination,Category,Origin,Destination_Coords,Destination_Long,Destination_Lat,Origin_Lat,Origin_Long,Distance
0,1,28.01.2022,Ebreichsdorf,CSN-B*,Zwentendorf,"[16.39163215125545, 47.96611095]",16.391632,47.966111,48.610603,16.43302,98.7359
1,2,11.02.2022,Stadl Paura,CSN-B*,Zwentendorf,"[13.8694007, 48.0838623]",13.869401,48.083862,48.610603,16.43302,258.7631
2,2,19.02.2022,Tulln an der Donau,CSN-C,Zwentendorf,"[16.0567744, 48.3311686]",16.056774,48.331169,48.610603,16.43302,55.9459
3,2,25.02.2022,Ebreichsdorf,CSN-B*,Zwentendorf,"[16.39163215125545, 47.96611095]",16.391632,47.966111,48.610603,16.43302,98.7359
4,2,25.02.2022,Stadl Paura,CSN-B*,Zwentendorf,"[13.8694007, 48.0838623]",13.869401,48.083862,48.610603,16.43302,258.7631
...,...,...,...,...,...,...,...,...,...,...,...
180,11,04.11.2022,Stadl Paura,CSN-B*,Zwentendorf,"[13.8694007, 48.0838623]",13.869401,48.083862,48.610603,16.43302,258.7631
181,11,11.11.2022,Stadl Paura,CSN-B*,Zwentendorf,"[13.8694007, 48.0838623]",13.869401,48.083862,48.610603,16.43302,258.7631
182,11,25.11.2022,Ebreichsdorf,CSN-B*,Zwentendorf,"[16.39163215125545, 47.96611095]",16.391632,47.966111,48.610603,16.43302,98.7359
183,12,02.12.2022,Stadl Paura,CSN-A*,Zwentendorf,"[13.8694007, 48.0838623]",13.869401,48.083862,48.610603,16.43302,258.7631


In [8]:
data = data[data["Category"] != "Drop"].reset_index(drop=True)

In [9]:
data

,Month,Date,Destination,Category,Origin,Destination_Coords,Destination_Long,Destination_Lat,Origin_Lat,Origin_Long,Distance
0,1,28.01.2022,Ebreichsdorf,CSN-B*,Zwentendorf,"[16.39163215125545, 47.96611095]",16.391632,47.966111,48.610603,16.43302,98.7359
1,2,11.02.2022,Stadl Paura,CSN-B*,Zwentendorf,"[13.8694007, 48.0838623]",13.869401,48.083862,48.610603,16.43302,258.7631
2,2,19.02.2022,Tulln an der Donau,CSN-C,Zwentendorf,"[16.0567744, 48.3311686]",16.056774,48.331169,48.610603,16.43302,55.9459
3,2,25.02.2022,Ebreichsdorf,CSN-B*,Zwentendorf,"[16.39163215125545, 47.96611095]",16.391632,47.966111,48.610603,16.43302,98.7359
4,2,25.02.2022,Stadl Paura,CSN-B*,Zwentendorf,"[13.8694007, 48.0838623]",13.869401,48.083862,48.610603,16.43302,258.7631
...,...,...,...,...,...,...,...,...,...,...,...
167,11,04.11.2022,Stadl Paura,CSN-B*,Zwentendorf,"[13.8694007, 48.0838623]",13.869401,48.083862,48.610603,16.43302,258.7631
168,11,11.11.2022,Stadl Paura,CSN-B*,Zwentendorf,"[13.8694007, 48.0838623]",13.869401,48.083862,48.610603,16.43302,258.7631
169,11,25.11.2022,Ebreichsdorf,CSN-B*,Zwentendorf,"[16.39163215125545, 47.96611095]",16.391632,47.966111,48.610603,16.43302,98.7359
170,12,02.12.2022,Stadl Paura,CSN-A*,Zwentendorf,"[13.8694007, 48.0838623]",13.869401,48.083862,48.610603,16.43302,258.7631


#### Transportation Cost

In [10]:
fuel_efficiency = 35/100
gasoline_price = 1.59

#Assumed that it is way back and forth -> time 2
data["Transportation Cost"] = 2 * fuel_efficiency * gasoline_price * data["Distance"]

### Competition Fee + Average Price (Revenue) + Horse Drain / Competition

In [11]:
data = pd.merge(data, compete_metrics, left_on="Category", right_on="cat")

In [12]:
data = data[[i for i in data.columns if i != "cat" and i != "Destination_Coords"]]

In [13]:
data

,Month,Date,Destination,Category,Origin,Destination_Long,Destination_Lat,Origin_Lat,Origin_Long,Distance,Transportation Cost,Entry Cost,Average Price,Horse Drain
0,1,28.01.2022,Ebreichsdorf,CSN-B*,Zwentendorf,16.391632,47.966111,48.610603,16.43302,98.7359,109.893057,200,100,-15
1,2,11.02.2022,Stadl Paura,CSN-B*,Zwentendorf,13.869401,48.083862,48.610603,16.43302,258.7631,288.003330,200,100,-15
2,2,25.02.2022,Ebreichsdorf,CSN-B*,Zwentendorf,16.391632,47.966111,48.610603,16.43302,98.7359,109.893057,200,100,-15
3,2,25.02.2022,Stadl Paura,CSN-B*,Zwentendorf,13.869401,48.083862,48.610603,16.43302,258.7631,288.003330,200,100,-15
4,3,18.03.2022,Stadl Paura,CSN-B*,Zwentendorf,13.869401,48.083862,48.610603,16.43302,258.7631,288.003330,200,100,-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,6,10.06.2022,Ebreichsdorf,CSN-A,Zwentendorf,16.391632,47.966111,48.610603,16.43302,98.7359,109.893057,250,500,-25
168,6,24.06.2022,Stadl Paura,CSN-A,Zwentendorf,13.869401,48.083862,48.610603,16.43302,258.7631,288.003330,250,500,-25
169,8,11.08.2022,Horitschon,CSN-A,Zwentendorf,16.547838,47.584833,48.610603,16.43302,161.1224,179.329231,250,500,-25
170,8,25.08.2022,Gniebing-Feldbach,CSN-A,Zwentendorf,15.853497,46.963234,48.610603,16.43302,236.4320,263.148816,250,500,-25


In [14]:
data.to_csv("./data/interim.csv",index=False)

In [16]:
data

,Month,Date,Destination,Category,Origin,Destination_Long,Destination_Lat,Origin_Lat,Origin_Long,Distance,Transportation Cost,Entry Cost,Average Price,Horse Drain
0,1,28.01.2022,Ebreichsdorf,CSN-B*,Zwentendorf,16.391632,47.966111,48.610603,16.43302,98.7359,109.893057,200,100,-15
1,2,11.02.2022,Stadl Paura,CSN-B*,Zwentendorf,13.869401,48.083862,48.610603,16.43302,258.7631,288.003330,200,100,-15
2,2,25.02.2022,Ebreichsdorf,CSN-B*,Zwentendorf,16.391632,47.966111,48.610603,16.43302,98.7359,109.893057,200,100,-15
3,2,25.02.2022,Stadl Paura,CSN-B*,Zwentendorf,13.869401,48.083862,48.610603,16.43302,258.7631,288.003330,200,100,-15
4,3,18.03.2022,Stadl Paura,CSN-B*,Zwentendorf,13.869401,48.083862,48.610603,16.43302,258.7631,288.003330,200,100,-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,6,10.06.2022,Ebreichsdorf,CSN-A,Zwentendorf,16.391632,47.966111,48.610603,16.43302,98.7359,109.893057,250,500,-25
168,6,24.06.2022,Stadl Paura,CSN-A,Zwentendorf,13.869401,48.083862,48.610603,16.43302,258.7631,288.003330,250,500,-25
169,8,11.08.2022,Horitschon,CSN-A,Zwentendorf,16.547838,47.584833,48.610603,16.43302,161.1224,179.329231,250,500,-25
170,8,25.08.2022,Gniebing-Feldbach,CSN-A,Zwentendorf,15.853497,46.963234,48.610603,16.43302,236.4320,263.148816,250,500,-25


### Model

In [183]:
data = pd.read_csv("./data/interim.csv")

In [184]:
data

,Month,Date,Destination,Category,Origin,Destination_Long,Destination_Lat,Origin_Lat,Origin_Long,Distance,Transportation Cost,Entry Cost,Average Price,Horse Drain
0,1,28.01.2022,Ebreichsdorf,CSN-B*,Zwentendorf,16.391632,47.966111,48.610603,16.43302,98.7359,109.893057,200,100,-15
1,2,11.02.2022,Stadl Paura,CSN-B*,Zwentendorf,13.869401,48.083862,48.610603,16.43302,258.7631,288.003330,200,100,-15
2,2,25.02.2022,Ebreichsdorf,CSN-B*,Zwentendorf,16.391632,47.966111,48.610603,16.43302,98.7359,109.893057,200,100,-15
3,2,25.02.2022,Stadl Paura,CSN-B*,Zwentendorf,13.869401,48.083862,48.610603,16.43302,258.7631,288.003330,200,100,-15
4,3,18.03.2022,Stadl Paura,CSN-B*,Zwentendorf,13.869401,48.083862,48.610603,16.43302,258.7631,288.003330,200,100,-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,6,10.06.2022,Ebreichsdorf,CSN-A,Zwentendorf,16.391632,47.966111,48.610603,16.43302,98.7359,109.893057,250,500,-25
168,6,24.06.2022,Stadl Paura,CSN-A,Zwentendorf,13.869401,48.083862,48.610603,16.43302,258.7631,288.003330,250,500,-25
169,8,11.08.2022,Horitschon,CSN-A,Zwentendorf,16.547838,47.584833,48.610603,16.43302,161.1224,179.329231,250,500,-25
170,8,25.08.2022,Gniebing-Feldbach,CSN-A,Zwentendorf,15.853497,46.963234,48.610603,16.43302,236.4320,263.148816,250,500,-25


In [185]:
#Get input from user how many horses he or she has
try:
    n = int(input("Number of Horse: \n"))
    
except ValueError:
    print("Please input an integer!")

Number of Horse: 
1


In [186]:
for i in range(1, n + 1):
    
    col = "Horse_"+str(i)
    
    data[col] = 0

In [187]:
data

,Month,Date,Destination,Category,Origin,Destination_Long,Destination_Lat,Origin_Lat,Origin_Long,Distance,Transportation Cost,Entry Cost,Average Price,Horse Drain,Horse_1
0,1,28.01.2022,Ebreichsdorf,CSN-B*,Zwentendorf,16.391632,47.966111,48.610603,16.43302,98.7359,109.893057,200,100,-15,0
1,2,11.02.2022,Stadl Paura,CSN-B*,Zwentendorf,13.869401,48.083862,48.610603,16.43302,258.7631,288.003330,200,100,-15,0
2,2,25.02.2022,Ebreichsdorf,CSN-B*,Zwentendorf,16.391632,47.966111,48.610603,16.43302,98.7359,109.893057,200,100,-15,0
3,2,25.02.2022,Stadl Paura,CSN-B*,Zwentendorf,13.869401,48.083862,48.610603,16.43302,258.7631,288.003330,200,100,-15,0
4,3,18.03.2022,Stadl Paura,CSN-B*,Zwentendorf,13.869401,48.083862,48.610603,16.43302,258.7631,288.003330,200,100,-15,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,6,10.06.2022,Ebreichsdorf,CSN-A,Zwentendorf,16.391632,47.966111,48.610603,16.43302,98.7359,109.893057,250,500,-25,0
168,6,24.06.2022,Stadl Paura,CSN-A,Zwentendorf,13.869401,48.083862,48.610603,16.43302,258.7631,288.003330,250,500,-25,0
169,8,11.08.2022,Horitschon,CSN-A,Zwentendorf,16.547838,47.584833,48.610603,16.43302,161.1224,179.329231,250,500,-25,0
170,8,25.08.2022,Gniebing-Feldbach,CSN-A,Zwentendorf,15.853497,46.963234,48.610603,16.43302,236.4320,263.148816,250,500,-25,0


In [188]:
header = data.columns

In [189]:
#Define decision variables
length = len(data.columns)

#Horses
horses = data.iloc[:,length - n:length].values

#Number of possible competitions per year
coms_per_year = 12 * 2 #2 competitions per month on average

#Max drain per month
drained = -50
drain = data["Horse Drain"].values

#Cost and Revenue per horse
revenue = data["Average Price"].values
cost = data["Entry Cost"].values

#Cost per competition
transport = data["Transportation Cost"].values

# Preparing an optimization model
model = ro.Model('Competition Allocator')

#h is for compete on non-compete, whereas y is for whether the manager sends any horses there or not
h = model.dvar((len(horses)), vtype="I")
y = model.dvar(len(horses), vtype="B")

In [190]:
drain

array([-15, -15, -15, -15, -15, -15, -15, -15, -15, -15, -15, -15, -15,
       -15, -15, -15, -15, -15, -15, -15, -15, -15, -15, -15, -15, -15,
       -15, -15, -15, -15, -15, -15, -15, -15, -15, -15, -15, -15, -15,
       -15, -15, -15, -15, -15, -15, -15, -15, -15, -15, -15, -15, -15,
       -15, -15, -15, -15, -15, -15, -15, -15, -15, -15, -10, -10, -10,
       -10, -10, -10, -10, -10, -10, -10, -10, -10, -10, -10, -10, -10,
       -10, -10, -10, -10, -10, -10, -10, -10, -10, -10, -10, -10, -10,
       -10, -10, -10, -10, -10, -10, -10, -10, -10, -10, -10, -10, -10,
       -10, -10, -10, -10, -10, -10, -10, -10, -10, -10, -10, -10, -10,
       -10, -10, -10, -10, -25, -25, -25, -25, -25, -25, -25, -25, -25,
       -25, -25, -25, -25, -25, -25, -25, -25, -25, -25, -25, -25, -25,
       -25, -25, -25, -25, -25, -25, -25, -25, -25, -25, -25, -15, -15,
       -15, -15, -15, -15, -15, -15, -15, -15, -30, -30, -25, -25, -25,
       -25, -25, -40])

In [191]:
#Objective
model.max(h@(revenue-cost) - transport@y)

#Contraints
model.st(
    h.sum() <= 24,
    sum((drain[i] * h[i] for i in range(length))) <= -100,
    10000*h[i] >= transport[i],
    h >= y,
    0 <= h <= 1
)

#Solve model
model.solve(solver)

Being solved by OR-Tools...
Solution status: 0
Running time: 0.0168s


In [192]:
#Decision variable inspection
h.get()

array([ 0.,  1.,  1.,  1.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,
        1.,  0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
       -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
       -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
       -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
       -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
       -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
       -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
       -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
       -0., -0., -0., -0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -0., -0.,
       -0., -0., -0., -0., -0., -0., -0., -0.,  1.,  1.,  0.,  0.,  0.,
        1.,  0.,  1.])

In [193]:
#Optimal payoff per year
model.get()

7100.0

### MapView to show where most locations will be

In [194]:
h = list(h.get())
data["Results"] = list(map(lambda x: int(x), h))
final = data[data["Results"] == 1].reset_index(drop=True)

In [195]:
final.head(5)

,Month,Date,Destination,Category,Origin,Destination_Long,Destination_Lat,Origin_Lat,Origin_Long,Distance,Transportation Cost,Entry Cost,Average Price,Horse Drain,Horse_1,Results
0,2,11.02.2022,Stadl Paura,CSN-B*,Zwentendorf,13.869401,48.083862,48.610603,16.43302,258.7631,288.003330,200,100,-15,0,1
1,2,25.02.2022,Ebreichsdorf,CSN-B*,Zwentendorf,16.391632,47.966111,48.610603,16.43302,98.7359,109.893057,200,100,-15,0,1
2,2,25.02.2022,Stadl Paura,CSN-B*,Zwentendorf,13.869401,48.083862,48.610603,16.43302,258.7631,288.003330,200,100,-15,0,1
3,3,18.03.2022,Stadl Paura,CSN-B*,Zwentendorf,13.869401,48.083862,48.610603,16.43302,258.7631,288.003330,200,100,-15,0,1
4,4,01.04.2022,Linz-Ebelsberg,CSN-B*,Zwentendorf,14.347104,48.251974,48.610603,16.43302,215.8538,240.245279,200,100,-15,0,1


In [196]:
#Initialize map at origin
first_row = final.iloc[1,:]
base_lat = first_row["Origin_Lat"]
base_long = first_row["Origin_Long"]
m = folium.Map(location=[base_lat, base_long], zoom_start=8)

#Add marker to origin
folium.Marker(
    [base_lat, base_long], popup="<i>{}</i>".format(" ".join([row["Category"],row["Destination"]])), tooltip=" ".join([row["Category"],row["Destination"]]),icon=folium.Icon(color="red")).add_to(m)

for index, row in final.iterrows():
    dest_lat = row["Destination_Lat"]
    dest_long = row["Destination_Long"]
    
    folium.Marker(
    [dest_lat, dest_long], popup="<i>{}</i>".format(" ".join([row["Category"],row["Destination"]])), tooltip=" ".join([row["Category"],row["Destination"]])
    ).add_to(m)
    
m